# **Analysis of Heart Disease Mortality Data**

### **Samarth Tuli, Ndenko Fontem, Yi Zhu, Kimia Samieinejad**    


**Importance of Heart Disease Mortality Research**    

**What is coronary heart disease?**

**Tutorial Purpose**

The objective of this tutorial is to evaluate many factors that may positively or negatively affect the heart disease mortality rates of populations across different countries so that we can get a better understanding of what factors should be focused upon most by the research community to reduce the overall risk of heart disease-related deaths globally. Data science is the right tool to achieve this because it will allow us to deconstruct complex heart disease mortality data into specific insights and recommendations that can be used by heart disease research leaders and policymakers to take immediate action through a 5-stage pipeline: data collection and processing, exploratory data analysis and visualizations, analysis/hypothesis testing/use of ML models, and insights & policy decisions.

In [5]:
print("Hello World. Testing notebook appears on GitHub page.")

Hello World. Testing notebook appears on GitHub page.


In [6]:
print("Yi Zhu testing")

Yi Zhu testing


In [ ]:
print("Ndenko testing")

In [30]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re




In [ ]:
# 3294 rows x 34 cols
suicide_rates= pd.read_csv('./data/suicide_rates_all_countries2012-2017.csv')
suicide_rates.tail()

In [28]:
# 1170 rows x 34 cols
blood_pressure = pd.read_csv('./data/mean_systolic_blood_pressure2012-2014.csv')
blood_pressure.tail()

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
1165,BP_06,Mean systolic blood pressure (age-standardized...,numeric,EUR,Europe,Country,SMR,San Marino,Year,2012,...,NaN,NaN,NaN,NaN,NaN,No data,827.0,No data available. Estimate could not be calc...,EN,2017-01-11T05:00:00.000Z
1166,BP_06,Mean systolic blood pressure (age-standardized...,numeric,AFR,Africa,Country,SSD,South Sudan,Year,2012,...,NaN,NaN,NaN,NaN,NaN,No data,827.0,No data available. Estimate could not be calc...,EN,2017-01-11T05:00:00.000Z
1167,BP_06,Mean systolic blood pressure (age-standardized...,numeric,AFR,Africa,Country,SSD,South Sudan,Year,2012,...,NaN,NaN,NaN,NaN,NaN,No data,827.0,No data available. Estimate could not be calc...,EN,2017-01-11T05:00:00.000Z
1168,BP_06,Mean systolic blood pressure (age-standardized...,numeric,EMR,Eastern Mediterranean,Country,SDN,Sudan,Year,2012,...,NaN,NaN,NaN,NaN,NaN,No data,827.0,No data available. Estimate could not be calc...,EN,2017-01-11T05:00:00.000Z
1169,BP_06,Mean systolic blood pressure (age-standardized...,numeric,EMR,Eastern Mediterranean,Country,SDN,Sudan,Year,2012,...,NaN,NaN,NaN,NaN,NaN,No data,827.0,No data available. Estimate could not be calc...,EN,2017-01-11T05:00:00.000Z


In [27]:
# 768 rows x 34 cols
blood_glucose = pd.read_csv('./data/fasting_blood_glucose_all_countries2012-2013.csv')
blood_glucose.tail()

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
763,NCD_GLUC_01,Mean fasting blood glucose (mmol/l) (age-stand...,numeric,WPR,Western Pacific,Country,PLW,Palau,Year,2012,...,NaN,NaN,6.42,NaN,7.29,6.86 [6.42 – 7.29],NaN,NaN,EN,2020-06-03T04:00:00.000Z
764,NCD_GLUC_01,Mean fasting blood glucose (mmol/l) (age-stand...,numeric,WPR,Western Pacific,Country,MHL,Marshall Islands,Year,2012,...,NaN,NaN,5.52,NaN,8.15,6.9 [5.52 – 8.15],NaN,NaN,EN,2020-06-03T04:00:00.000Z
765,NCD_GLUC_01,Mean fasting blood glucose (mmol/l) (age-stand...,numeric,WPR,Western Pacific,Country,FSM,Micronesia (Federated States of),Year,2012,...,NaN,NaN,6.52,NaN,7.55,7.04 [6.52 – 7.55],NaN,NaN,EN,2020-05-29T04:00:00.000Z
766,NCD_GLUC_01,Mean fasting blood glucose (mmol/l) (age-stand...,numeric,WPR,Western Pacific,Country,FSM,Micronesia (Federated States of),Year,2012,...,NaN,NaN,6.73,NaN,7.84,7.28 [6.73 – 7.84],NaN,NaN,EN,2020-06-03T04:00:00.000Z
767,NCD_GLUC_01,Mean fasting blood glucose (mmol/l) (age-stand...,numeric,WPR,Western Pacific,Country,TON,Tonga,Year,2012,...,NaN,NaN,6.78,NaN,7.77,7.28 [6.78 – 7.77],NaN,NaN,EN,2020-06-03T04:00:00.000Z


In [26]:
# 9360 rows x 34 cols
bmi = pd.read_csv('./data/bmi_all_countries2012-2015.csv')
bmi.tail()

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
9355,NCD_BMI_MEANC,Mean BMI (kg/m&#xb2;) (crude estimate),numeric,EMR,Eastern Mediterranean,Country,SDN,Sudan,Year,2012,...,NaN,NaN,NaN,NaN,NaN,No data,827.0,No data available.� Estimate could not be calc...,EN,2017-09-27T04:00:00.000Z
9356,NCD_BMI_MEANC,Mean BMI (kg/m&#xb2;) (crude estimate),numeric,EMR,Eastern Mediterranean,Country,SDN,Sudan,Year,2012,...,NaN,NaN,NaN,NaN,NaN,No data,827.0,No data available.� Estimate could not be calc...,EN,2017-09-28T04:00:00.000Z
9357,NCD_BMI_MEANC,Mean BMI (kg/m&#xb2;) (crude estimate),numeric,EMR,Eastern Mediterranean,Country,SDN,Sudan,Year,2012,...,NaN,NaN,NaN,NaN,NaN,No data,827.0,No data available.� Estimate could not be calc...,EN,2017-09-28T04:00:00.000Z
9358,NCD_BMI_MEANC,Mean BMI (kg/m&#xb2;) (crude estimate),numeric,EMR,Eastern Mediterranean,Country,SDN,Sudan,Year,2012,...,NaN,NaN,NaN,NaN,NaN,No data,827.0,No data available.� Estimate could not be calc...,EN,2017-09-28T04:00:00.000Z
9359,NCD_BMI_MEANC,Mean BMI (kg/m&#xb2;) (crude estimate),numeric,EMR,Eastern Mediterranean,Country,SDN,Sudan,Year,2012,...,NaN,NaN,NaN,NaN,NaN,No data,827.0,No data available.� Estimate could not be calc...,EN,2017-09-27T04:00:00.000Z
